In [0]:
import numpy
from time import time

In [0]:
class Constants:
  
  SIGMOID = "sigmoid"

In [0]:
class Neuron:
  
  # name of the activation function
  act_func = None 
  in_edges = None
  out_edges = None
  # output of a neuron i.e activation(summation(w_i*x_i))
  output = None       
  # error at a neuron during back propagation
  local_error = None  
  # input to the input-layer-neuron
  input_datum = None   
  # loss for output-layer-neuron at the end of forward propagation
  forward_prop_loss = None   

In [0]:
class Neuron(Neuron):
  
  # returns output of activation function
  def apply_act_func(self, inp):
    if(self.act_func == Constants.SIGMOID):
      return ( 1/ ( 1+numpy.exp(-1*inp) ) )

In [0]:
class Neuron(Neuron):
  
  # returns derivative of activation function
  def apply_act_func_der(self, inp):
    if(self.act_func == Constants.SIGMOID):
      return (inp*(1-inp))

In [0]:
class Neuron(Neuron):
  
  # forward propagation (computes output of a neuron)
  # assumption-1: outputs of start_neurons of all self.in_edges are calculated
  # assumption-2: self.input_datum is initialized for a first-layered-neuron
  def forward_prop(self):
    # first-layered-neuron
    if(self.in_edges == None): 
      self.output = self.input_datum
      return
    output = 0.0
    ind = 0
    # compute summation(w_i*x_i)
    while(ind < len(self.in_edges)):
      edge = self.in_edges[ind]
      weight = edge.weight
      inp = edge.start_neuron.output
      output = output + (weight*inp)
      ind = ind + 1
    # apply activation function
    self.output = self.apply_act_func(output)

In [0]:
class Neuron(Neuron):
  
  # back propagation (updates the weights of in_edges)
  # local_error: before multiplying by weight
  # assumption-1: local_errors of all end_neurons of self.out_edges are calculated
  # assumption-2: self.output is computed
  # assumption-3: self.forward_prop_loss is initialized for an output-layered-neuron
  def back_prop(self, local_error):
    # output-layered-neuron
    if(self.out_edges == None): 
      local_error_self = self.forward_prop_loss
    else:
      ind = 0
      local_error_self = 0.0
      # compute self.local_error
      while(ind < len(self.out_edges)):
        edge = self.out_edges[ind]
        weight = edge.weight
        local_error_other = edge.end_neuron.local_error
        local_error_self = local_error_self + (local_error_other * weight)
        ind = ind + 1
    act_func_der = self.apply_act_func_der(self.output)
    self.local_error = local_error_self * act_func_der
    # update self.in_edges
    ind = 0
    while(ind < len(self.in_edges)):
      edge = self.in_edges[ind]
      inp = edge.start_neuron.output
      update = self.local_error * inp
      self.in_edges[ind].weight = edge.weight + update
      ind = ind + 1

In [0]:
# An edge between 2 neurons
class Edge:
  
  start_neuron = None
  end_neuron = None
  weight = None

In [0]:
# Fully connected Layer
class Layer:
  
  tot_neurons = None
  act_func = None
  neurons = None
  prev_layer = None
  next_layer = None

In [0]:
class Layer(Layer):
  
  # add in_edges to cur_layer's neurons & out_edges to prev_layer's neurons
  # assumption: should have non-None value to self.prev_layer from 2nd layer onwards
  def build(self):
    # cur refers to current layer
    cur_neuron_ind = 0 
    self.neurons = []
    # add neurons
    while(cur_neuron_ind < self.tot_neurons):
      cur_neuron = Neuron()
      cur_neuron.act_func = self.act_func
      # add in_edges to cur_neuron
      if(self.prev_layer != None):
        cur_neuron.in_edges = []
        prev_neuron_ind = 0
        while(prev_neuron_ind < self.prev_layer.tot_neurons):
          prev_neuron = self.prev_layer.neurons[prev_neuron_ind]
          edge = Edge
          edge.start_neuron = prev_neuron
          edge.end_neuron = cur_neuron
          edge.weight = 0.5
          cur_neuron.in_edges.append(edge)
          prev_neuron_ind = prev_neuron_ind + 1
      self.neurons.append(cur_neuron)
      cur_neuron_ind = cur_neuron_ind + 1
    # add out_edges to prev_layer's neurons
    if(self.prev_layer != None):
      prev_neuron_ind = 0
      while(prev_neuron_ind < self.prev_layer.tot_neurons):
        prev_neuron = self.prev_layer.neurons[prev_neuron_ind]
        prev_neuron.out_edges = []
        cur_neuron_ind = 0
        # add out_edges to prev_neuron
        while(cur_neuron_ind < self.tot_neurons):
          cur_neuron = self.neurons[cur_neuron_ind]
          edge = Edge
          edge.start_neuron = prev_neuron
          edge.end_neuron = cur_neuron
          edge.weight = 0.5
          prev_neuron.out_edges.append(edge)
          cur_neuron_ind = cur_neuron_ind + 1
        prev_neuron_ind = prev_neuron_ind + 1
    
      

In [0]:
class Neural_Network:
  
  # including input, hidden and output layers
  tot_layers = None
  # a list containing no. of neurons in each layer
  tot_neurons = None
  # single activation func; ex: Constants.SIGMOID
  act_func = None
  # a list containing actual layers
  layers = None
  

In [0]:
class Neural_Network(Neural_Network):
  
  def build(self):
    cur_layer_ind = 0
    prev_layer_ind = -1
    self.layers = []
    while(cur_layer_ind < self.tot_layers):
      cur_layer = Layer()
      cur_layer.tot_neurons = self.tot_neurons[cur_layer_ind]
      cur_layer.act_func = self.act_func
      if(prev_layer_ind != -1):
        cur_layer.prev_layer = self.layers[prev_layer_ind]
      cur_layer.build()
      self.layers.append(cur_layer)
      cur_layer_ind = cur_layer_ind + 1
      prev_layer_ind = prev_layer_ind + 1

In [0]:
class Neural_Network(Neural_Network):
  
  def print(self):
    print("layers: " + str(self.tot_layers))
    print("neurons: " + str(self.tot_neurons))
    print("activation: " + str(self.act_func))
    print("=======================")
    print("layer.neuron.edge: weight")
    layer_ind = 0
    while(layer_ind < self.tot_layers):
      cur_layer = self.layers[layer_ind]
      neuron_ind = 0
      while(neuron_ind < cur_layer.tot_neurons):
        cur_neuron = cur_layer.neurons[neuron_ind]
        if(cur_neuron.out_edges != None):
          out_edge_ind = 0
          while(out_edge_ind < len(cur_neuron.out_edges)):
            out_edge = cur_neuron.out_edges[out_edge_ind]
            print(str(layer_ind) + "." + str(neuron_ind) + "." + str(out_edge_ind) + ": " + str(out_edge.weight))
            out_edge_ind = out_edge_ind + 1
        neuron_ind = neuron_ind + 1
      layer_ind = layer_ind + 1
      if(layer_ind != self.tot_layers):
        print("=======================")

In [0]:
class Main:
  
  @staticmethod
  def main():
    nn = Neural_Network()
    nn.tot_layers = 3
    nn.tot_neurons = [2, 4, 1]
    nn.act_func = Constants.SIGMOID
    nn.build()
    nn.print()    

In [131]:
start_time = time()
Main.main()
end_time = time()
elapsed_time = end_time - start_time
print("processed in " + str(elapsed_time) + " seconds.")

layers: 3
neurons: [2, 4, 1]
activation: sigmoid
layer.neuron.edge: weight
0.0.0: 0.5
0.0.1: 0.5
0.0.2: 0.5
0.0.3: 0.5
0.1.0: 0.5
0.1.1: 0.5
0.1.2: 0.5
0.1.3: 0.5
1.0.0: 0.5
1.1.0: 0.5
1.2.0: 0.5
1.3.0: 0.5
processed in 0.004755973815917969 seconds.
